<a href="https://colab.research.google.com/github/tylaar1/PICAR-autopilot/blob/main/CNN_DUAL_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SWITCH TO **`T4 GPU`** OR THE **`HPC`**

# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt

2025-04-29 14:06:46.013659: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-29 14:06:46.109476: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-29 14:06:46.110675: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-29 14:06:46.967592: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# makes it so pd dfs aren't truncated

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [45]:
#from google.colab import drive
#drive.mount('/content/drive')

# 1) DATA PRE-PROCESSING

a) Load in labels + image file paths

b) combine them into one dataframe

c) EDA - spotted and removed erroneous label (speed = 1.42...)

- `cleaned_df` is the cleaned df with a) b) c) completed

d) convert images to numerical RGB feature maps - ML algorithms only understand numerical data

e) Splitting data into training and validation sets

f) data augmentation applied to training set

### 1a) load in labels + image file paths

In [3]:
#labels_file_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/training_norm.csv' # tylers file path
#labels_file_path = '/home/apyba3/KAGGLEDATAmachine-learning-in-science-ii-2025/training_norm.csv' # ben hpc file path (mlis2 cluster)
labels_file_path = '/home/ppytr13/machine-learning-in-science-ii-2025/training_norm.csv' # tyler hpc file path (mlis2 cluster)
labels_df = pd.read_csv(labels_file_path, index_col='image_id')

In [4]:
#image_folder_path = '/home/apyba3/KAGGLEDATAmachine-learning-in-science-ii-2025/training_data/training_data' # bens hpc file path
#image_folder_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/training_data/training_data' # tylers file path
image_folder_path = '/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data' # tyler hpc file path
image_file_paths = [
    os.path.join(image_folder_path, f)
    for f in os.listdir(image_folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
]

image_file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0])) # sorts the files in the right order (1.png, 2.png, 3.png, ...)

imagefilepaths_df = pd.DataFrame(
    image_file_paths,
    columns=['image_file_paths'],
    index=[int(os.path.splitext(os.path.basename(path))[0]) for path in image_file_paths]
)

imagefilepaths_df.index.name = 'image_id'

Checking labels dataframe

In [48]:
labels_df.head()

,angle,speed
image_id,,
1,0.4375,0.0
2,0.8125,1.0
3,0.4375,1.0
4,0.6250,1.0
5,0.5000,0.0


Checking image file paths dataframe - as you can see the file paths are ordered correctly (1.png, 2.png, 3.png, ...)

In [49]:
imagefilepaths_df.head()

,image_file_paths
image_id,
1,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/1.png
2,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/2.png
3,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3.png
4,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/4.png
5,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/5.png


### 1b) Combine labels and image file paths into one dataframe

In [5]:
merged_df = pd.merge(labels_df, imagefilepaths_df, on='image_id', how='inner')
merged_df['speed'] = merged_df['speed'].round(6) # to get rid of floating point errors

In [6]:
merged_df.head()

,angle,speed,image_file_paths
image_id,,,
1,0.4375,0.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/1.png
2,0.8125,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/2.png
3,0.4375,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3.png
4,0.6250,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/4.png
5,0.5000,0.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/5.png


In [52]:
merged_df.loc[3139:3143]

,angle,speed,image_file_paths
image_id,,,
3139,0.750,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3139.png
3140,0.875,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3140.png
3142,0.625,0.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3142.png
3143,0.625,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3143.png


The above cell shows that:

 1) the image files and labels match (see image_id and the number at the end of the file path)

 2) the missing rows in labels_df (image_id: 3141, 3999, 4895, 8285, 10171) have been taken care of

### 1c) EDA

In [7]:
merged_df.value_counts('angle')

angle
0.7500    2123
0.5000    2046
0.6875    2007
0.6250    1963
0.5625    1609
0.4375    1467
0.8125    1147
0.3750     428
0.8750     301
0.3125     213
0.2500     104
0.1250      99
0.1875      98
0.9375      65
0.0000      60
1.0000      35
0.0625      28
Name: count, dtype: int64

note: imbalance datset

identifying the row with the erroneous speed value

In [54]:
merged_df[merged_df['speed'] == 1.428571]

,angle,speed,image_file_paths
image_id,,,
3884,0.4375,1.428571,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3884.png


we want to remove this row

In [8]:
cleaned_df = merged_df[merged_df['speed'] != 1.428571]
cleaned_df.loc[3882:3886]

,angle,speed,image_file_paths
image_id,,,
3882,0.5625,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3882.png
3883,0.3750,0.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3883.png
3885,0.0000,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3885.png
3886,0.7500,1.0,/home/ppytr13/machine-learning-in-science-ii-2025/training_data/training_data/3886.png


### 1d) convert images to numerical RGB feature maps

In [9]:
def process_image(image_path, resized_shape=(300, 224)):
    # Load and preprocess the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, resized_shape)
    image = image / 255.0  # Normalize pixel values to [0,1]
    return image

# Creating the dataset
dataset = tf.data.Dataset.from_tensor_slices(
    (cleaned_df["image_file_paths"], cleaned_df["angle"], cleaned_df["speed"])
)  # Convert pandas DataFrame to a tf.data.Dataset

# Apply the map function to process images and format labels
dataset = dataset.map(
    lambda image_path, angle, speed: (
        process_image(image_path),  # Process the image
        {"classification": angle, "regression": speed}  # Format labels as a dictionary
    )
)

dataset = dataset.cache()
dataset = dataset.shuffle(len(cleaned_df))
dataset = dataset.batch(32)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

2025-04-29 14:13:18.113594: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


lets check and see if what we have done works

### 1e) Splitting data into training and validation sets (test set is already provided in kaggle data)

In [10]:
# 80-20 split

dataset_size = tf.data.experimental.cardinality(dataset).numpy()
train_size = int(0.8 * dataset_size)

train_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)

In [58]:
print(f"Train size: {train_size}, validation size: {dataset_size - train_size}")

Train size: 344, validation size: 87


In [59]:
validation_dataset

<_SkipDataset element_spec=(TensorSpec(shape=(None, 300, 224, 3), dtype=tf.float32, name=None), {'classification': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'regression': TensorSpec(shape=(None,), dtype=tf.float64, name=None)})>

### 1f) Data augmentation applied to training set

Flipping or rotating the image will render the angle labels incorrect so none of that was applied to the images for this regression task

- Random Brightness Adjustment
- Random Contrast Adjustment
- Random Hue Adjustment
- Random Saturation Adjustment


In [11]:
def augment_image(image, label):
  seed = (6, 9)
  image = tf.image.stateless_random_brightness(image, 0.2, seed)
  image = tf.image.stateless_random_contrast(image, 0.8, 1.2, seed)
  image = tf.image.stateless_random_hue(image, 0.2, seed)
  image = tf.image.stateless_random_saturation(image, 0.8, 1.2, seed)
  return image, label

augmented_dataset = train_dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.concatenate(augmented_dataset)
train_dataset = train_dataset.shuffle(buffer_size=len(cleaned_df))

# 2) Model Building - MobileNetV3Large Transfer Learning

a) Set up model architecture

b) define training step

c) training the model on the training set

d) fine-tuning

### 2a) Set up model architecture

- MobileNetV2 to learn lower level features
- global average pooling layer
- drop out layer
- dense layer with sigmoid activation

In [12]:
dropoutrate = 0.2
num_classes = 1

In [15]:
input_layer = tf.keras.Input(shape=(300, 224, 3))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = tf.keras.layers.MaxPooling2D((2, 2))(x) #experiment with removing this
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(dropoutrate)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(dropoutrate)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)

#split outputs to predict speed and angle
classification_output = tf.keras.layers.Dense(num_classes, activation='sigmoid', name="classification")(x)
regression_output = tf.keras.layers.Dense(1, activation='linear', name="regression")(x)

#combine both outputs
model = tf.keras.Model(inputs=input_layer, outputs=[classification_output, regression_output])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss={'classification': 'binary_crossentropy', 'regression': 'mse'},
              metrics={'classification': 'accuracy', 'regression': 'mse'})


model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 300, 224, 3)]        0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 298, 222, 32)         896       ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_1 (MaxPoolin  (None, 149, 111, 32)         0         ['conv2d_1[0][0]']            
 g2D)                                                                                             
                                                                                                  
 global_average_pooling2d_1  (None, 32)                   0         ['max_pooling2d_1[0][0]'

### 2c) Training the model on the training set

In [16]:
history = model.fit(train_dataset,
                    epochs=5,
                    batch_size=32,
                    validation_data=validation_dataset)

Epoch 1/5


2025-04-29 14:14:07.529956: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 2474 of 13792
2025-04-29 14:14:17.532916: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 5117 of 13792
2025-04-29 14:14:27.521631: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 6845 of 13792
2025-04-29 14:14:37.521851: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 9619 of 13792
2025-04-29 14:14:47.518646: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 12760 of 13792
2025-04-29 14:14:53.702808: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shuffle buffer filled.
2025-04-29 14:14:53.713272: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while)

688/688 [==============================] - 330s 386ms/step - loss: 0.8681 - classification_loss: 0.6736 - regression_loss: 0.1946 - classification_accuracy: 0.0026 - regression_mse: 0.1946 - val_loss: 0.8509 - val_classification_loss: 0.6723 - val_regression_loss: 0.1786 - val_classification_accuracy: 0.0022 - val_regression_mse: 0.1786
Epoch 2/5
688/688 [==============================] - 258s 366ms/step - loss: 0.8536 - classification_loss: 0.6708 - regression_loss: 0.1828 - classification_accuracy: 0.0025 - regression_mse: 0.1828 - val_loss: 0.8414 - val_classification_loss: 0.6704 - val_regression_loss: 0.1710 - val_classification_accuracy: 0.0025 - val_regression_mse: 0.1710
Epoch 3/5
688/688 [==============================] - 256s 365ms/step - loss: 0.8323 - classification_loss: 0.6699 - regression_loss: 0.1624 - classification_accuracy: 0.0024 - regression_mse: 0.1624 - val_loss: 0.8162 - val_classification_loss: 0.6713 - val_regression_loss: 0.1449 - val_classification_accuracy:

: 

In [64]:
#model.save_weights('/home/apyba3/cnn.weights.h5')
model.save_weights('/home/ppytr13/cnn.weights.h5')


# 3) Test-Set Predictions

a) load in test data

b) convert test images to numerical RGB feature maps

c) generate predictions on the test set

d) correctly format the predictions into a pandas dataframe

e) save predictions to a file inside the hpc (to then later send from hpc to my laptop)

### 3a) load in test data

In [65]:
#image_folder_path = '/home/apyba3/KAGGLEDATAmachine-learning-in-science-ii-2025/test_data/test_data'
#image_folder_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/test_data/test_data'
image_folder_path = '/home/ppytr13/machine-learning-in-science-ii-2025/test_data/test_data'
image_file_paths = [
    os.path.join(image_folder_path, f)
    for f in os.listdir(image_folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
]

image_file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0])) # sorts the files in the right order (1.png, 2.png, 3.png, ...)

imagefilepaths_df = pd.DataFrame(
    image_file_paths,
    columns=['image_file_paths'],
    index=[int(os.path.splitext(os.path.basename(path))[0]) for path in image_file_paths]
)

imagefilepaths_df.index.name = 'image_id'
imagefilepaths_df.head()

,image_file_paths
image_id,
1,/home/ppytr13/machine-learning-in-science-ii-2025/test_data/test_data/1.png
2,/home/ppytr13/machine-learning-in-science-ii-2025/test_data/test_data/2.png
3,/home/ppytr13/machine-learning-in-science-ii-2025/test_data/test_data/3.png
4,/home/ppytr13/machine-learning-in-science-ii-2025/test_data/test_data/4.png
5,/home/ppytr13/machine-learning-in-science-ii-2025/test_data/test_data/5.png


### 3b) convert test images to numerical RGB feature maps

In [66]:
def process_image_no_label(image_path, resized_shape=(300, 224)): #match real images
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Use decode_png for PNG images
    image = tf.image.resize(image, resized_shape)  # Resize to uniform shape
    image = image / 255.0  # Normalize pixel values to [0,1]
    return image

test_dataset = tf.data.Dataset.from_tensor_slices((imagefilepaths_df["image_file_paths"]))

test_dataset = test_dataset.map(process_image_no_label, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

### 3c) generate predictions on test set

In [67]:
predictions = model.predict(test_dataset.take(1))

1/1 [==============================] - 0s 206ms/step


### 3d) correctly format the predictions into a pandas dataframe

In [68]:
predictions_array = np.concatenate([predictions[0], predictions[1]], axis=1)
predictions_df = pd.DataFrame(predictions_array, columns=['angle', 'speed'])

In [69]:
predictions_df.head()


,angle,speed
0,0.599836,0.454232
1,0.627595,0.965500
2,0.588144,0.925471
3,0.605792,0.859938
4,0.611401,0.971044


In [70]:
predictions_df.loc[predictions_df['speed'] > 0.5, 'speed'] = 1
predictions_df.loc[predictions_df['speed'] <= 0.5, 'speed'] = 0
predictions_df['speed'] = predictions_df['speed'].astype(int)
predictions_df.head()

,angle,speed
0,0.599836,0
1,0.627595,1
2,0.588144,1
3,0.605792,1
4,0.611401,1


In [71]:
predictions_df['angle'].value_counts()

angle
0.599836    1
0.627595    1
0.608784    1
0.601144    1
0.591117    1
0.608221    1
0.599727    1
0.602319    1
0.603321    1
0.606676    1
0.590413    1
0.590589    1
0.594540    1
0.573855    1
0.602705    1
0.603262    1
0.611849    1
0.621432    1
0.587323    1
0.605662    1
0.582879    1
0.598381    1
0.603941    1
0.612062    1
0.608005    1
0.620282    1
0.599721    1
0.595299    1
0.611401    1
0.605792    1
0.588144    1
0.595021    1
Name: count, dtype: int64

### 3e) save predictions to a file inside the hpc (to then later send from hpc to my laptop)

In [72]:
#predictions_df.to_csv('/home/apyba3/mbnetv3_angleregression_predictions.csv')
predictions_df.to_csv('/home/ppytr13/cnn_dual_predictions.csv')

## instead - convert to tf lite

In [73]:
import tensorflow as tf

# Define the converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable default optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Specify fixed input shape
converter._experimental_fixed_input_shape = {"serving_default_input": [1, 300, 244, 3]}  # Batch size 1

# Use FP16 for smaller model size and faster inference
converter.target_spec.supported_types = [tf.float16]

# Convert the model
tflite_model = converter.convert()

# Save the model as a TFLite file
tflite_model_path = '/home/ppytr13/cnn_dual_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print("Optimized TFLite model saved at:", tflite_model_path)

INFO:tensorflow:Assets written to: /tmp/tmp9kmeo694/assets


INFO:tensorflow:Assets written to: /tmp/tmp9kmeo694/assets


Optimized TFLite model saved at: /home/ppytr13/cnn_dual_model.tflite


2025-04-01 11:45:03.458107: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-04-01 11:45:03.458303: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-04-01 11:45:03.459963: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp9kmeo694
2025-04-01 11:45:03.461780: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-04-01 11:45:03.461806: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp9kmeo694
2025-04-01 11:45:03.469455: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-04-01 11:45:03.518188: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmp9kmeo694
2025-04-01 11:45:03.532428: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 72464 m